<h1 style="text-align: center;">A Tutorial for (Probabilistic) Bayesian Neural Networks</h1>
<p style="text-align: center;"> A short version of <a href="https://keras.io/examples/keras_recipes/bayesian_neural_networks/">Khalid Salama's blog post</a> with Son Hai Le's modifications </p>

**Goal of this document**
- Implementation of standard neural network ([SNN](http://www.deeplearningbook.org))
- Implementation of 2 types of Bayesian neural network ([BNN](https://ieeexplore.ieee.org/document/9756596)):
    * BNN: deal with [epistemic uncertainty](https://www.sciencedirect.com/science/article/abs/pii/S0167473008000556)
    * Probabilistic BNN: deal with both [aleatoric uncertainty](https://www.sciencedirect.com/science/article/abs/pii/S0167473008000556) and [epistemic uncertainty](https://www.sciencedirect.com/science/article/abs/pii/S0167473008000556)


**What are different compared to the original** [Khalid Salama's blog post](https://keras.io/examples/keras_recipes/bayesian_neural_networks)?
- Make the document more succinct, remove unnecessary theory parts
- Add an [exploratory data analysis](https://en.wikipedia.org/wiki/Exploratory_data_analysis)
- Add codes to save and load the weights of the model
- Add visualization of the predictions
- Divide the [original document](https://keras.io/examples/keras_recipes/bayesian_neural_networks) into 2 parts:
    * Part 1: training
    * Part 2: prediction and visualization

*Note: follow [hyperlinks]() to see more details*

## Introduction

Taking a probabilistic approach to deep learning allows to account for *uncertainty*,
so that models can assign less levels of confidence to incorrect predictions.
Sources of uncertainty can be found in the data, due to measurement error or
noise in the labels, or the model, due to insufficient data availability for
the model to learn effectively.


This example demonstrates how to build basic probabilistic Bayesian neural networks
to account for these two types of uncertainty.
We use [TensorFlow Probability](https://www.tensorflow.org/probability) library,
which is compatible with Keras API.

This example requires TensorFlow 2.3 or higher.
You can install Tensorflow Probability using the following command:

```python
pip install tensorflow-probability
```

## The dataset

We use the [Wine Quality](https://archive.ics.uci.edu/ml/datasets/wine+quality)
dataset, which is available in the [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/wine_quality).
We use the red wine subset, which contains 4,898 examples.
The dataset has 11numerical physicochemical features of the wine, and the task
is to predict the wine quality, which is a score between 0 and 10.
In this example, we treat this as a regression task.

You can install TensorFlow Datasets using the following command:

```python
pip install tensorflow-datasets
```

## Setup

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

## Create training and evaluation datasets

Here, we load the `wine_quality` dataset using `tfds.load()`, and we convert
the target feature to float. Then, we shuffle the dataset and split it into
training and test sets. We take the first `train_size` examples as the train
split, and the rest as the test split.

In [2]:

def get_train_and_test_splits(train_size, batch_size=1):
    # We prefetch with a buffer the same size as the dataset because th dataset
    # is very small and fits into memory.
    dataset = (
        tfds.load(name="wine_quality", as_supervised=True, split="train")
        .map(lambda x, y: (x, tf.cast(y, tf.float32)))
        .prefetch(buffer_size=dataset_size)
        .cache()
    )
    # We shuffle with a buffer the same size as the dataset.
    train_dataset = (
        dataset.take(train_size).shuffle(buffer_size=train_size).batch(batch_size)
    )
    test_dataset = dataset.skip(train_size).batch(batch_size)

    return train_dataset, test_dataset


## Compile, train, and evaluate the model

In [3]:
hidden_units = [8, 8]
learning_rate = 0.001


def run_experiment(model, loss, train_dataset, test_dataset):

    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
        loss=loss,
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    print("Start training the model...")
    model.fit(train_dataset, epochs=num_epochs, validation_data=test_dataset)
    print("Model training finished.")
    _, rmse = model.evaluate(train_dataset, verbose=0)
    print(f"Train RMSE: {round(rmse, 3)}")

    print("Evaluating model performance...")
    _, rmse = model.evaluate(test_dataset, verbose=0)
    print(f"Test RMSE: {round(rmse, 3)}")


## Create model inputs

In [4]:
FEATURE_NAMES = [
    "fixed acidity",
    "volatile acidity",
    "citric acid",
    "residual sugar",
    "chlorides",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "pH",
    "sulphates",
    "alcohol",
]


def create_model_inputs():
    inputs = {}
    for feature_name in FEATURE_NAMES:
        inputs[feature_name] = layers.Input(
            name=feature_name, shape=(1,), dtype=tf.float32
        )
    return inputs


## Experiment 1: standard neural network

We create a standard deterministic neural network model as a baseline.

In [5]:

def create_baseline_model():
    inputs = create_model_inputs()
    input_values = [value for _, value in sorted(inputs.items())]
    features = keras.layers.concatenate(input_values)
    features = layers.BatchNormalization()(features)

    # Create hidden layers with deterministic weights using the Dense layer.
    for units in hidden_units:
        features = layers.Dense(units, activation="sigmoid")(features)
    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


Let's split the wine dataset into training and test sets, with 85% and 15% of
the examples, respectively.

In [6]:
dataset_size = 4898
batch_size = 256
train_size = int(dataset_size * 0.85)
train_dataset, test_dataset = get_train_and_test_splits(train_size, batch_size)

Now let's train the baseline model. We use the `MeanSquaredError`
as the loss function.

In [7]:
num_epochs = 100
mse_loss = keras.losses.MeanSquaredError()
baseline_model = create_baseline_model()
run_experiment(baseline_model, mse_loss, train_dataset, test_dataset)
baseline_model.save_weights('baseline_model.h5')


Start training the model...
Epoch 1/100
17/17 [==============================] - 3s 29ms/step - loss: 52.2458 - root_mean_squared_error: 7.2281 - val_loss: 52.9195 - val_root_mean_squared_error: 7.2746
Epoch 2/100
17/17 [==============================] - 0s 11ms/step - loss: 49.7624 - root_mean_squared_error: 7.0542 - val_loss: 50.3065 - val_root_mean_squared_error: 7.0927
Epoch 3/100
17/17 [==============================] - 0s 12ms/step - loss: 47.7104 - root_mean_squared_error: 6.9073 - val_loss: 47.9372 - val_root_mean_squared_error: 6.9237
Epoch 4/100
17/17 [==============================] - 0s 12ms/step - loss: 45.7824 - root_mean_squared_error: 6.7663 - val_loss: 45.7087 - val_root_mean_squared_error: 6.7608
Epoch 5/100
17/17 [==============================] - 0s 13ms/step - loss: 43.9513 - root_mean_squared_error: 6.6296 - val_loss: 43.6203 - val_root_mean_squared_error: 6.6046
Epoch 6/100
17/17 [==============================] - 0s 13ms/step - loss: 42.2123 - root_mean_squared_

We take a sample from the test set use the model to obtain predictions for them.
Note that since the baseline model is deterministic, we get a single a
*point estimate* prediction for each test example, with no information about the
uncertainty of the model nor the prediction.

In [8]:
sample = 10
examples, targets = list(test_dataset.unbatch().shuffle(batch_size * 10).batch(sample))[
    0
]

predicted = baseline_model(examples).numpy()
for idx in range(sample):
    print(f"Predicted: {round(float(predicted[idx][0]), 1)} - Actual: {targets[idx]}")

Predicted: 6.2 - Actual: 7.0
Predicted: 6.2 - Actual: 6.0
Predicted: 5.2 - Actual: 5.0
Predicted: 6.0 - Actual: 5.0
Predicted: 6.3 - Actual: 6.0
Predicted: 6.1 - Actual: 6.0
Predicted: 5.3 - Actual: 6.0
Predicted: 6.3 - Actual: 6.0
Predicted: 5.5 - Actual: 6.0
Predicted: 5.6 - Actual: 5.0


## Experiment 2: Bayesian neural network (BNN)

The object of the Bayesian approach for modeling neural networks is to capture
the *epistemic uncertainty*, which is uncertainty about the model fitness,
due to limited training data.

The idea is that, instead of learning specific weight (and bias) *values* in the
neural network, the Bayesian approach learns weight *distributions*
- from which we can sample to produce an output for a given input -
to encode weight uncertainty.

Thus, we need to define prior and the posterior distributions of these weights,
and the training process is to learn the parameters of these distributions.

In [9]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model


We use the `tfp.layers.DenseVariational` layer instead of the standard
`keras.layers.Dense` layer in the neural network model.

In [10]:

def create_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # The output is deterministic: a single point estimate.
    outputs = layers.Dense(units=1)(features)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


The epistemic uncertainty can be reduced as we increase the size of the
training data. That is, the more data the BNN model sees, the more it is certain
about its estimates for the weights (distribution parameters).
Let's test this behaviour by training the BNN model on a small subset of
the training set, and then on the full training set, to compare the output variances.

### Train BNN  with a small training subset.

In [11]:
num_epochs = 500
train_sample_size = int(train_size * 0.3)
small_train_dataset = train_dataset.unbatch().take(train_sample_size).batch(batch_size)

bnn_model_small = create_bnn_model(train_sample_size)
run_experiment(bnn_model_small, mse_loss, small_train_dataset, test_dataset)
bnn_model_small.save_weights('bnn_model_small.h5')

Start training the model...
Epoch 1/500
5/5 [==============================] - 3s 338ms/step - loss: 45.0909 - root_mean_squared_error: 6.7142 - val_loss: 41.6101 - val_root_mean_squared_error: 6.4495
Epoch 2/500
5/5 [==============================] - 0s 28ms/step - loss: 42.8748 - root_mean_squared_error: 6.5469 - val_loss: 40.2341 - val_root_mean_squared_error: 6.3420
Epoch 3/500
5/5 [==============================] - 0s 26ms/step - loss: 41.0221 - root_mean_squared_error: 6.4037 - val_loss: 38.3371 - val_root_mean_squared_error: 6.1906
Epoch 4/500
5/5 [==============================] - 0s 27ms/step - loss: 40.8104 - root_mean_squared_error: 6.3871 - val_loss: 41.0549 - val_root_mean_squared_error: 6.4064
Epoch 5/500
5/5 [==============================] - 0s 27ms/step - loss: 42.6077 - root_mean_squared_error: 6.5264 - val_loss: 41.0855 - val_root_mean_squared_error: 6.4088
Epoch 6/500
5/5 [==============================] - 0s 37ms/step - loss: 39.0140 - root_mean_squared_error: 6.24

Since we have trained a BNN model, the model produces a different output each time
we call it with the same input, since each time a new set of weights are sampled
from the distributions to construct the network and produce an output.
The less certain the mode weights are, the more variability (wider range) we will
see in the outputs of the same inputs.

In [12]:

def compute_predictions(model, iterations=100):
    predicted = []
    for _ in range(iterations):
        predicted.append(model(examples).numpy())
    predicted = np.concatenate(predicted, axis=1)

    prediction_mean = np.mean(predicted, axis=1).tolist()
    prediction_min = np.min(predicted, axis=1).tolist()
    prediction_max = np.max(predicted, axis=1).tolist()
    prediction_range = (np.max(predicted, axis=1) - np.min(predicted, axis=1)).tolist()

    for idx in range(sample):
        print(
            f"Predictions mean: {round(prediction_mean[idx], 2)}, "
            f"min: {round(prediction_min[idx], 2)}, "
            f"max: {round(prediction_max[idx], 2)}, "
            f"range: {round(prediction_range[idx], 2)} - "
            f"Actual: {targets[idx]}"
        )


compute_predictions(bnn_model_small)

Predictions mean: 6.21, min: 5.85, max: 6.47, range: 0.62 - Actual: 7.0
Predictions mean: 6.29, min: 5.76, max: 6.52, range: 0.76 - Actual: 6.0
Predictions mean: 5.08, min: 4.05, max: 5.66, range: 1.61 - Actual: 5.0
Predictions mean: 5.98, min: 5.42, max: 6.3, range: 0.88 - Actual: 5.0
Predictions mean: 6.32, min: 5.87, max: 6.52, range: 0.65 - Actual: 6.0
Predictions mean: 6.08, min: 5.37, max: 6.41, range: 1.04 - Actual: 6.0
Predictions mean: 5.56, min: 4.5, max: 5.97, range: 1.46 - Actual: 6.0
Predictions mean: 6.28, min: 5.87, max: 6.46, range: 0.59 - Actual: 6.0
Predictions mean: 5.36, min: 4.2, max: 5.97, range: 1.76 - Actual: 6.0
Predictions mean: 5.6, min: 4.81, max: 6.03, range: 1.22 - Actual: 5.0


### Train BNN  with the whole training set.

In [13]:
num_epochs = 500
bnn_model_full = create_bnn_model(train_size)
run_experiment(bnn_model_full, mse_loss, train_dataset, test_dataset)
bnn_model_full.save_weights('bnn_model_full.h5')
compute_predictions(bnn_model_full)

Start training the model...
Epoch 1/500
17/17 [==============================] - 2s 46ms/step - loss: 24.9390 - root_mean_squared_error: 4.9934 - val_loss: 20.0372 - val_root_mean_squared_error: 4.4758
Epoch 2/500
17/17 [==============================] - 0s 16ms/step - loss: 24.4136 - root_mean_squared_error: 4.9406 - val_loss: 24.3498 - val_root_mean_squared_error: 4.9341
Epoch 3/500
17/17 [==============================] - 0s 15ms/step - loss: 21.9507 - root_mean_squared_error: 4.6847 - val_loss: 21.4075 - val_root_mean_squared_error: 4.6264
Epoch 4/500
17/17 [==============================] - 0s 14ms/step - loss: 21.2156 - root_mean_squared_error: 4.6055 - val_loss: 20.6904 - val_root_mean_squared_error: 4.5482
Epoch 5/500
17/17 [==============================] - 0s 15ms/step - loss: 20.5078 - root_mean_squared_error: 4.5281 - val_loss: 18.0289 - val_root_mean_squared_error: 4.2453
Epoch 6/500
17/17 [==============================] - 0s 15ms/step - loss: 18.4130 - root_mean_squared_

Notice that the model trained with the full training dataset shows smaller range
(uncertainty) in the prediction values for the same inputs, compared to the model
trained with a subset of the training dataset.

## Experiment 3: probabilistic Bayesian neural network

So far, the output of the standard and the Bayesian NN models that we built is
deterministic, that is, produces a point estimate as a prediction for a given example.
We can create a probabilistic NN by letting the model output a distribution.
In this case, the model captures the *aleatoric uncertainty* as well,
which is due to irreducible noise in the data, or to the stochastic nature of the
process generating the data.

In this example, we model the output as a `IndependentNormal` distribution,
with learnable mean and variance parameters. If the task was classification,
we would have used `IndependentBernoulli` with binary classes, and `OneHotCategorical`
with multiple classes, to model distribution of the model output.

In [14]:

def create_probablistic_bnn_model(train_size):
    inputs = create_model_inputs()
    features = keras.layers.concatenate(list(inputs.values()))
    features = layers.BatchNormalization()(features)

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in hidden_units:
        features = tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        )(features)

    # Create a probabilisticå output (Normal distribution), and use the `Dense` layer
    # to produce the parameters of the distribution.
    # We set units=2 to learn both the mean and the variance of the Normal distribution.
    distribution_params = layers.Dense(units=2)(features)
    outputs = tfp.layers.IndependentNormal(1)(distribution_params)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


Since the output of the model is a distribution, rather than a point estimate,
we use the [negative loglikelihood](https://en.wikipedia.org/wiki/Likelihood_function)
as our loss function to compute how likely to see the true data (targets) from the
estimated distribution produced by the model.

In [15]:

def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)


num_epochs = 1000
prob_bnn_model = create_probablistic_bnn_model(train_size)
run_experiment(prob_bnn_model, negative_loglikelihood, train_dataset, test_dataset)
prob_bnn_model.save_weights('prob_bnn_model.h5')

Start training the model...
Epoch 1/1000
17/17 [==============================] - 3s 70ms/step - loss: 49.1691 - root_mean_squared_error: 6.0751 - val_loss: 38.3142 - val_root_mean_squared_error: 6.0983
Epoch 2/1000
17/17 [==============================] - 0s 19ms/step - loss: 41.7781 - root_mean_squared_error: 6.2082 - val_loss: 29.4559 - val_root_mean_squared_error: 6.2254
Epoch 3/1000
17/17 [==============================] - 0s 15ms/step - loss: 29.7613 - root_mean_squared_error: 6.0704 - val_loss: 24.2351 - val_root_mean_squared_error: 6.1507
Epoch 4/1000
17/17 [==============================] - 0s 16ms/step - loss: 30.0575 - root_mean_squared_error: 5.9745 - val_loss: 25.9842 - val_root_mean_squared_error: 5.7415
Epoch 5/1000
17/17 [==============================] - 0s 16ms/step - loss: 25.2751 - root_mean_squared_error: 5.9883 - val_loss: 24.5888 - val_root_mean_squared_error: 5.9560
Epoch 6/1000
17/17 [==============================] - 0s 15ms/step - loss: 20.1535 - root_mean_sq

Now let's produce an output from the model given the test examples.
The output is now a distribution, and we can use its mean and variance
to compute the confidence intervals (CI) of the prediction.

In [16]:
prediction_distribution = prob_bnn_model(examples)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(sample):
    print(
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {targets[idx]}"
    )

Prediction mean: 6.39, stddev: 0.81, 95% CI: [7.98 - 4.79] - Actual: 7.0
Prediction mean: 6.44, stddev: 0.82, 95% CI: [8.06 - 4.83] - Actual: 6.0
Prediction mean: 5.24, stddev: 0.65, 95% CI: [6.52 - 3.96] - Actual: 5.0
Prediction mean: 5.83, stddev: 0.72, 95% CI: [7.24 - 4.41] - Actual: 5.0
Prediction mean: 6.53, stddev: 0.84, 95% CI: [8.17 - 4.88] - Actual: 6.0
Prediction mean: 6.21, stddev: 0.77, 95% CI: [7.72 - 4.69] - Actual: 6.0
Prediction mean: 5.41, stddev: 0.67, 95% CI: [6.73 - 4.09] - Actual: 6.0
Prediction mean: 6.44, stddev: 0.83, 95% CI: [8.06 - 4.81] - Actual: 6.0
Prediction mean: 5.37, stddev: 0.67, 95% CI: [6.7 - 4.05] - Actual: 6.0
Prediction mean: 5.44, stddev: 0.67, 95% CI: [6.76 - 4.12] - Actual: 5.0
